In [19]:
import pandas as pd
import numpy as np

In [20]:
varE = "Demanda de carne Kg"
df = pd.DataFrame({varE: ['240-250', '250-260', '260-280', '280-300'],"f":[7,10,6,8]})

df["F"] = df["f"].cumsum()
df[["Li", "Ls"]] = df[varE].str.split("-", expand=True).astype(int)
df["A"] = df["Ls"] - df["Li"]
df["Pm"] = (df["Ls"] + df["Li"]) / 2


print(df)

  Demanda de carne Kg   f   F   Li   Ls   A     Pm
0             240-250   7   7  240  250  10  245.0
1             250-260  10  17  250  260  10  255.0
2             260-280   6  23  260  280  20  270.0
3             280-300   8  31  280  300  20  290.0


In [21]:
promedio = (df["f"] * df["Pm"]).sum() / df["f"].sum()
varianza = (df["f"] * ((df["Pm"] - promedio) ** 2)).sum() / (df["f"].sum() - 1)
desviacion = varianza**0.5
Ca = desviacion / promedio
estado = "homogeneos" if Ca < 0.20 else "heterogeneos"

print(f"El promedio es {round(promedio,4)} tn/ha")
print(f"La varianza es {round(varianza,4)}")
print(f"La desviacion es {round(desviacion, 4)} tn/ha")
print(f"El coeficiente de variacion es {round(Ca*100, 4)}%")
print(f"Los datos son {estado} por lo que es una medida significativa.")

El promedio es 264.6774 tn/ha
La varianza es 298.2258
La desviacion es 17.2692 tn/ha
El coeficiente de variacion es 6.5246%
Los datos son homogeneos por lo que es una medida significativa.


In [22]:
indiceClaseModal = df["f"].idxmax()
claseAnterior = df.loc[indiceClaseModal-1]
claseModal = df.loc[indiceClaseModal]
claseSiguiente = df.loc[indiceClaseModal+1]

moda = claseModal["Li"] + claseModal["A"] * (claseModal["f"] - claseAnterior["f"]) / ((claseModal["f"] -
                                                                                       claseAnterior["f"]) + (claseModal["f"] - claseSiguiente["f"]))

masPersonal = f'La moda es {round(moda,4)} por lo que tomara mas personal' if moda > 255 else f'La moda es {
    round(moda, 4)} kg por lo que no tomara mas personal'



print(masPersonal)

La moda es 254.2857 kg por lo que no tomara mas personal


In [23]:
def calcularPercentil(k: int, df: pd.DataFrame):
  frecuencia = (k / 100) * df["f"].sum()
  intervalo_percentil = df[df["F"] >= frecuencia].iloc[0]

  percentil = intervalo_percentil["Li"] + (
      frecuencia - intervalo_percentil["F"] + intervalo_percentil["f"]) / intervalo_percentil["f"] * intervalo_percentil["A"]

  return percentil


def calcularInverso(valor, df: pd.DataFrame):
    # Buscar el intervalo
    ia = None
    ie = None
    for index, row in df.iterrows():
        if row['Li'] <= valor <= row['Ls']:
            ie = row
            if index != 0:
              ia = df.loc[index-1]['F']
            elif index == 0:
                ia = 0
            break
    final = (valor - ie['Li']) / \
        (ie['Ls'] - ie['Li'])
    final = final * ie['f']
    final = final + ia
    final = final * 100
    final = final / df['F'].max()
    return final

In [24]:
print(f"La maxima cantidad de carne demandada el 30% de los dias de menor demanda es {round(calcularPercentil(30,df),4)} kg")

La maxima cantidad de carne demandada el 30% de los dias de menor demanda es 252.3 kg


In [25]:
print(f"El porcentaje de dias que se demandan mas de 285 kg es {round(100-calcularInverso(285,df),4)}%")

El porcentaje de dias que se demandan mas de 285 kg es 19.3548%


In [26]:
asimetria = "asimetria positiva por lo que podemos decir que la demanda es mayor al promedio" if promedio > moda else "asimetria negativa por lo que podemos decir que la demanda es menor al promedio"

print("El rendimiento posee una", asimetria)

El rendimiento posee una asimetria positiva por lo que podemos decir que la demanda es mayor al promedio
